In [7]:
import pandas as pd
from dotenv import load_dotenv
from sqlalchemy import create_engine
import os
import candlestick_patterns_v2 as cp

In [19]:

# Load environment variables from .env file
load_dotenv()
# Access the database URL
db_url = os.getenv('DB_URL')

# Create connect sql database
def create_connection_database(db_url):
    # Database connection string
    db_url = db_url
    # Create an SQLAlchemy engine
    return create_engine(db_url)

# read data from the PostgreSQL table
try:
   with create_connection_database(db_url).connect() as connection:
    table_name = "nifty_top_500_stocks"
    query = f"SELECT * from {table_name};"
    df = pd.read_sql_query(query, con=create_connection_database(db_url))
    print("data pulled from sql table")
except Exception as e:
    print(f"Error: {e}")

data pulled from sql table


In [20]:
def sma(dataframe,column_name,timeperiod):
    dataframe[f"SMA_{timeperiod}"] = dataframe[column_name].rolling(window=timeperiod).mean()
    return dataframe

def ema(dataframe,column_name,timeperiod):
    dataframe[f"EMA_{timeperiod}"] = dataframe[column_name].ewm(span=timeperiod, adjust=False).mean()
    return dataframe

In [10]:
df.SYMBOL.unique()
df_symbol = df[df.SYMBOL == "360ONE"].sort_values(by="DATE")
df_symbol.tail()

,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
0,2024-12-20,EQ,1225.35,1260.00,1210.35,1225.35,1223.15,1249.50,1250.26,1270.00,591.5,1452337,1.815797e+09,53665,360ONE
122121,2024-12-23,EQ,1238.05,1280.75,1208.95,1249.50,1225.10,1232.70,1229.79,1280.75,591.5,499836,6.146931e+08,40886,360ONE
123826,2024-12-24,EQ,1230.00,1239.70,1201.20,1232.70,1233.40,1236.35,1226.88,1280.75,591.5,284990,3.496481e+08,20952,360ONE
126157,2024-12-26,EQ,1236.40,1312.10,1214.25,1236.35,1245.00,1256.25,1234.71,1312.10,591.5,1129000,1.393993e+09,45199,360ONE
127273,2024-12-27,EQ,1262.85,1276.95,1221.90,1256.25,1252.35,1250.90,1241.41,1312.10,591.5,827586,1.027370e+09,43131,360ONE


In [30]:
# Volume confirmation
def volume_confirmation(days,df):
    """Check if current volume exceeds 10-day average volume."""
    df['Avg_Volume'] = df['VOLUME'].rolling(window=days).mean()
    df['Volume_Confirmed'] = df['VOLUME'] > 1.1 * df['Avg_Volume']
    return df 


def process_dataframe(df):
    patterns = {
        "Doji": [],
        "Four Price Doji": [],
        "Long Legged Doji": [],
        "Gravestone Doji": [],
        "Dragonfly Doji": [],
        "Hammer": [],
        "Hanging Man": [],
        "Bullish Marubozu": [],
        "Bearish Marubozu": [],
    }

    for _, row in df.iterrows():
        candle = cp.SingleCandlePattern(row)
        patterns["Doji"].append(candle.is_doji())
        patterns["Four Price Doji"].append(candle.is_four_price_doji())
        patterns["Long Legged Doji"].append(candle.is_long_legged_doji())
        patterns["Gravestone Doji"].append(candle.is_gravestone_doji())
        patterns["Dragonfly Doji"].append(candle.is_dragonfly_doji())
        patterns["Hammer"].append(candle.is_hammer())
        patterns["Hanging Man"].append(candle.is_hanging_man())
        patterns["Bullish Marubozu"].append(candle.is_bullish_murubozu())
        patterns["Bearish Marubozu"].append(candle.is_bearish_murubozu())

    # Add results to the DataFrame
    for pattern, results in patterns.items():
        df[pattern] = results
    
    single_candle_patterns_columns = ["Doji",
        "Four Price Doji",
        "Long Legged Doji",
        "Gravestone Doji",
        "Dragonfly Doji",
        "Hammer",
        "Hanging Man",
        "Bullish Marubozu",
        "Bearish Marubozu"]
    df = df.sort_values(by=['SYMBOL','DATE'])
    df['Single_Candlestick_pattern'] = df[single_candle_patterns_columns].any(axis=1)
    df = volume_confirmation(10,df)
    return df

In [31]:
processed_df = process_dataframe(df)
display(processed_df.head())

TypeError: Series.sort_values() got an unexpected keyword argument 'by'

In [13]:
df.sort_values()

(129646, 15)

In [15]:
df.DATE.min().date()

datetime.date(2023, 12, 22)

In [17]:
df_date = df[df['DATE']==df.DATE.min()]
df_date

,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,52W L,VOLUME,VALUE,NO OF TRADES,SYMBOL
247,2023-12-22,EQ,661.00,664.85,647.05,635.70,655.90,657.90,654.22,1960.00,395.10,611023,3.997459e+08,25691,360ONE
495,2023-12-22,EQ,31468.95,31563.55,30993.00,31358.30,31232.00,31182.00,31215.34,33250.00,21300.45,1812,5.656219e+07,1063,3MINDIA
743,2023-12-22,EQ,4723.00,4765.00,4686.10,4684.50,4753.00,4759.00,4735.44,4957.00,2640.00,155535,7.365260e+08,22442,ABB
991,2023-12-22,EQ,2120.00,2122.95,2079.00,2107.40,2090.00,2090.75,2105.04,2645.95,1592.35,444653,9.360122e+08,20838,ACC
1239,2023-12-22,EQ,3513.00,3581.00,3497.10,3513.00,3570.00,3574.30,3557.63,3830.00,2383.50,94756,3.371067e+08,17036,AIAENG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121128,2023-12-22,EQ,266.20,272.65,263.75,262.20,267.30,267.55,269.16,299.70,170.10,18959049,5.102983e+09,88524,ZEEL
121376,2023-12-22,EQ,633.00,639.95,604.10,624.90,619.05,620.05,620.80,639.95,201.50,3298618,2.047773e+09,78978,ZENSARTECH
121624,2023-12-22,EQ,130.55,131.00,125.30,127.35,127.90,128.50,128.74,131.75,44.35,63113800,8.125102e+09,187775,ZOMATO
121872,2023-12-22,EQ,675.00,689.40,673.05,671.45,679.00,679.95,682.09,696.50,404.60,2027527,1.382961e+09,41760,ZYDUSLIFE


In [18]:
processed_df = process_dataframe(df_date)
processed_df = processed_df[processed_df['Single_Candlestick_pattern']==True]
processed_df

C:\Users\megha\AppData\Local\Temp\ipykernel_21808\962826550.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[pattern] = results
C:\Users\megha\AppData\Local\Temp\ipykernel_21808\962826550.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[pattern] = results
C:\Users\megha\AppData\Local\Temp\ipykernel_21808\962826550.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the doc

,DATE,SERIES,OPEN,HIGH,LOW,PREV. CLOSE,LTP,CLOSE,VWAP,52W H,...,Doji,Four Price Doji,Long Legged Doji,Gravestone Doji,Dragonfly Doji,Hammer,Hanging Man,Bullish Marubozu,Bearish Marubozu,Single_Candlestick_pattern
247,2023-12-22,EQ,661.00,664.85,647.05,635.70,655.90,657.90,654.22,1960.00,...,True,False,True,False,False,False,True,False,False,True
1239,2023-12-22,EQ,3513.00,3581.00,3497.10,3513.00,3570.00,3574.30,3557.63,3830.00,...,False,False,False,False,False,False,False,True,False,True
1487,2023-12-22,EQ,1588.00,1602.60,1575.05,1585.70,1590.00,1588.15,1588.66,1800.00,...,True,False,False,False,False,False,False,False,False,True
2135,2023-12-22,EQ,599.00,602.35,593.20,593.20,597.45,598.55,598.31,635.00,...,True,False,True,False,False,False,False,False,False,True
2631,2023-12-22,EQ,22367.30,22661.00,22324.60,22354.75,22600.00,22604.00,22525.51,24744.25,...,False,False,False,False,False,False,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119144,2023-12-22,EQ,950.00,954.45,943.00,942.65,947.25,947.60,949.14,957.15,...,True,False,True,False,False,False,False,False,True,True
119640,2023-12-22,EQ,146.45,147.80,142.50,144.65,145.30,145.30,144.51,169.95,...,False,False,False,False,False,False,True,False,False,True
119888,2023-12-22,EQ,820.00,828.00,813.55,817.75,821.00,821.25,818.74,1024.95,...,True,False,False,False,False,False,False,False,False,True
120136,2023-12-22,EQ,1360.00,1372.00,1353.50,1356.00,1363.00,1359.25,1364.44,1733.00,...,True,False,False,False,False,False,False,False,False,True
